In [21]:
!pip install requests
!pip install xmltodict
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 3.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 25.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3


In [2]:
import geopandas
import json
import shapely
import shapely.geometry
import xarray
import rasterio as rio
import rioxarray
import os
import fiona
import nasa_hls
import urllib.request as urlreq
import pandas as pd
import numpy as np
import requests
import xmltodict
import shutil
import datetime

from shapely.ops import transform
from shapely.geometry import Point
from shapely.geometry import Polygon
from pystac_client import Client 
from collections import defaultdict
from glob import glob
from rasterio.enums import Resampling
from rasterio import Affine
from rasterio.crs import CRS
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline

In [3]:
##### START OPTIONS #####
yoi = [2021]
cloud_thres = 5
valid_months = [3,4,5,6,7,8,9]
root_path = "/home/gfm-gap-filling-td/t3_data/"

## file paths
# spath = root_path + f"CDL_HLS_dataframe{yoi[0]}"
# image_index_file = root_path + f"image_index{yoi[0]}"
chip_file =  root_path + "chip_bbox.geojson"
# chip_csv = root_path + "chip_tracker.csv"
kml_file = root_path + 'sentinel_tile_grid.kml'
# cdl_reclass_csv = root_path + "cdl_freq.csv"
# tile_tracker_csv = root_path + "tile_tracker.csv"

## folder paths
# hdf_dir = root_path + "hdf/"
chip_dir = root_path + 'chips/'
tif_dir = root_path + 'tif/'
# chip_dir_binary = root_path + 'chips_binary/'
chip_dir_multi = root_path + 'chips_multi/'

chip_dir_filt = root_path + 'chips_filtered/'
# chip_dir_binary_filt = root_path + 'chips_binary_filtered/'
chip_dir_multi_filt = root_path + 'chips_multi_filtered/'

chip_fmask_dir = root_path + 'chips_fmask/'

#####  END OPTIONS  #####

In [4]:
with open(chip_file, "r") as file:
    chips = json.load(file)
    
chip_ids = []
chip_x = []
chip_y = []
for item in chips['features']:
    #print(item)
    chip_ids.append(item['properties']['id'])
    chip_x.append(item['properties']['center'][0])
    chip_y.append(item['properties']['center'][1])

In [5]:
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
tile_src.head(5)

,Name,Description,geometry,minx,miny,maxx,maxy
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,-180.0,-73.064633,180.0,-72.012478
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-83.835334,180.0,-82.796720
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.939452,180.0,-81.906947
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.044055,180.0,-81.016439
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-81.148070,180.0,-80.124456


In [6]:
def find_tile(x,y):
# Identify closest tile
    s = (tile_x - x)**2+(tile_y - y)**2
    tname = tile_name[np.argmin(s)]
    return(tname)

In [7]:
chip_df = pd.DataFrame({"chip_id" : chip_ids, "chip_x" : chip_x, "chip_y" : chip_y})
chip_df['tile'] = chip_df.apply(lambda row : find_tile(row['chip_x'], row['chip_y']), axis = 1)
chip_df.tail(5)

,chip_id,chip_x,chip_y,tile
9095,069_125,-91.273691,29.693950,15RXN
9096,069_126,-91.274726,29.633329,15RXN
9097,069_127,-91.275758,29.572708,15RXN
9098,069_128,-91.276786,29.512086,15RXN
9099,069_129,-91.277812,29.451463,15RXN


In [8]:
chips['features'][9099]['geometry']

{'type': 'MultiPolygon',
 'coordinates': [[[[-91.31194315793084, 29.482218836228824],
    [-91.31294658186975, 29.42159483294573],
    [-91.24370141095643, 29.420699714104572],
    [-91.24265685208046, 29.481321513467094],
    [-91.31194315793084, 29.482218836228824]]]]}

In [9]:
tiles = chip_df.tile.unique().tolist()
tiles[0:5]

['14SQG', '14SQF', '14SQE', '14SQD', '14SQC']

In [ ]:
## go through list(item_collectio) and get only those item ids that are in current tile
## get list of item ids that have 5% or less cloud cover

## ask mike
## spatial coverage filtering (100%, 90%, 80%.... 50%)
## find first, middle, last images by time

## ask steve about s3 access
## download those images, B02, B03, B04, B8A, Fmask

In [10]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')

In [11]:
tile_list = []
print(f"There are a total of {len(tiles)} tiles")
tile_iter = 0
for current_tile in tiles:

    chip_df_filt = chip_df.loc[chip_df.tile == current_tile]#.reset_index()
    first_chip_id = chip_df_filt.chip_id.iloc[0]
    first_chip_index_in_json = chip_ids.index(first_chip_id)
    roi = chips['features'][first_chip_index_in_json]['geometry']

    search = catalog.search(
        collections = ['HLSS30.v2.0'],
        intersects = roi,
        datetime = '2022-03-01/2022-09-30',
    ) 
    
    num_results = search.matched()
    item_collection = search.get_all_items()
    
    tile_name = "T" + current_tile
    iter_items = 0
    print(f"Gathering information for tile {tile_name}")
    print(f"There are a total of {num_results} matching tiles for chip_{first_chip_id}'s geometry")
    
    for i in tqdm(item_collection):
        if i.id.split('.')[2] == tile_name:
            if i.properties['eo:cloud_cover'] <= cloud_thres:
                response = requests.get(i.assets['metadata'].href)
                if response.status_code == 200:
                    temp_xml = response.text
                    temp_xml = xmltodict.parse(temp_xml)
                    # print(temp_xml['Granule']['AdditionalAttributes']['AdditionalAttribute'][3]['Values']['Value'])
                    temp_dict = {"tile_id": tile_name, "cloud_cover": i.properties['eo:cloud_cover'],
                                 "date": datetime.datetime.strptime(i.properties['datetime'].split('T')[0], "%Y-%m-%d"), 
                                 "spatial_cover": int(temp_xml['Granule']['AdditionalAttributes']['AdditionalAttribute'][3]['Values']['Value']),
                                 "links": {"B02": i.assets['B02'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://'), 
                                           "B03": i.assets['B03'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://'), 
                                           "B04": i.assets['B04'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://'), 
                                           "B8A": i.assets['B8A'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://'),
                                           "B11": i.assets['B11'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://'),
                                           "B12": i.assets['B12'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://'),
                                           "Fmask": i.assets['Fmask'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')}}
                    tile_list.append(temp_dict)
                    iter_items += 1
                else: 
                    assert False, f"Failed to fetch XML from {i.assets['metadata'].href}. Error code: {response.status_code}"

    tile_iter += 1
    print(f"Information for tile {tile_name} is collected, a total of {iter_items} out of {num_results} tiles pass the filter ({tile_iter}/{len(tiles)})")

    
tile_df = pd.DataFrame(tile_list)

There are a total of 60 tiles
Gathering information for tile T14SQG
There are a total of 69 matching tiles for chip_000_000's geometry


100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [00:06<00:00, 10.88it/s]


Information for tile T14SQG is collected, a total of 11 out of 69 tiles pass the filter (1/60)
Gathering information for tile T14SQF
There are a total of 141 matching tiles for chip_000_005's geometry


100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [00:05<00:00, 27.07it/s]


Information for tile T14SQF is collected, a total of 12 out of 141 tiles pass the filter (2/60)
Gathering information for tile T14SQE
There are a total of 279 matching tiles for chip_000_020's geometry


...

Information for tile T15RXQ is collected, a total of 17 out of 303 tiles pass the filter (58/60)
Gathering information for tile T15RXP
There are a total of 150 matching tiles for chip_060_109's geometry


100%|███████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 154.79it/s]


Information for tile T15RXP is collected, a total of 4 out of 150 tiles pass the filter (59/60)
Gathering information for tile T15RXN
There are a total of 155 matching tiles for chip_060_124's geometry


100%|███████████████████████████████████████████████████████████████████████████████| 155/155 [00:01<00:00, 148.38it/s]

Information for tile T15RXN is collected, a total of 3 out of 155 tiles pass the filter (60/60)


In [12]:
tile_df[0:5]

,tile_id,cloud_cover,date,spatial_cover,links
0,T14SQG,0,2022-03-15,99,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
1,T14SQG,1,2022-04-09,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
2,T14SQG,1,2022-04-14,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
3,T14SQG,0,2022-06-13,99,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
4,T14SQG,0,2022-06-28,99,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...


In [24]:
tile_df.to_csv("/home/gfm-gap-filling-td/t3_data/tile_df.csv", index=False)

In [13]:
def spatial_filtering (dataframe):
    """
        Using spatial coverage percentage to filter chips

        Args:
            dataframe: A pandas dataframe that generated previously
    """
    cover_list = [100, 90, 80, 70, 60, 50]
    tile_list_ft = []
    tile_list = dataframe.tile_id.unique().tolist()
    
    for tile in tqdm(tile_list):
        temp_df = dataframe[dataframe.tile_id == tile]
        for cover_pct in cover_list:
            
            temp_df_filtered = temp_df[temp_df.spatial_cover >= cover_pct]
            if len(temp_df_filtered) >= 3:
                for i in range(len(temp_df_filtered)):
                    tile_list_ft.append(temp_df_filtered.iloc[i])
                break
    
    tile_df_filtered = pd.DataFrame(tile_list_ft)
    return tile_df_filtered

In [14]:
# check_spatial_cover(tile_df, 90)
cover_df = spatial_filtering(tile_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 735.85it/s]


In [15]:
def select_scenes(dataframe):
    select_tiles = []
    tile_list = dataframe.tile_id.unique().tolist()

    for tile in tqdm(tile_list):
        temp_df = dataframe[dataframe.tile_id == tile].sort_values('date').reset_index(drop=True)
        select_tiles.extend([temp_df.iloc[0], temp_df.iloc[len(temp_df) // 2], temp_df.iloc[-1]])

    return pd.DataFrame(select_tiles).reset_index(drop=True)

In [17]:
selected_tiles = select_scenes(cover_df)

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1267.79it/s]


{'B02': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.B02.tif',
 'B03': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.B03.tif',
 'B04': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.B04.tif',
 'B8A': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.B8A.tif',
 'B11': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.B11.tif',
 'B12': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.B12.tif',
 'Fmask': 's3://lp-prod-protected/HLSS30.020/HLS.S30.T14SQG.2022099T170849.v2.0/HLS.S30.T14SQG.2022099T170849.v2.0.Fmask.tif'}

In [20]:
selected_tiles.head(12)

,tile_id,cloud_cover,date,spatial_cover,links
0,T14SQG,1,2022-04-09,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
1,T14SQG,0,2022-07-23,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
2,T14SQG,0,2022-09-11,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
3,T14SQF,1,2022-04-09,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
4,T14SQF,0,2022-07-13,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
5,T14SQF,1,2022-08-02,100,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
6,T14SQE,0,2022-03-20,93,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
7,T14SQE,1,2022-06-13,93,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
8,T14SQE,0,2022-09-26,93,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...
9,T14SQD,0,2022-03-02,91,{'B02': 's3://lp-prod-protected/HLSS30.020/HLS...


In [23]:
selected_tiles.to_csv("/home/gfm-gap-filling-td/t3_data/selected_tiles.csv", index=False)

In [ ]:
s3_cred_endpoint = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'

In [ ]:
def get_temp_creds():
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url).json()

In [ ]:
temp_creds_req = get_temp_creds()
#temp_creds_req                      # !!! BEWARE, removing the # on this line will print your temporary S3 credentials.

In [ ]:
session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

In [ ]:
rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()